### Connect to a PostGIS database

In [1]:
import psycopg2

In [2]:
# help(psycopg2.connect)

In [3]:
credentials = {
  'database': 'postgis', 
  'user': 'masteruser', 
  'password': 'connecttoplotly',
  'port': 5432,
  'host': 'test-postgres.cwwxgcilxwxw.us-west-2.rds.amazonaws.com'
}

In [4]:
connection = psycopg2.connect(user=credentials['user'], 
                  			password=credentials['password'], 
                  			database=credentials['database'], 
                  			host=credentials['host'], 
                  			port=credentials['port'])

In [5]:
cursor = connection.cursor()

In [6]:
connection.autocommit

False

means we have to remember to commit our changes at the end

### Now that we have the database cursor, let's create the table scaffold

[postgis docs](http://postgis.net/docs/using_postgis_dbmanagement.html)

In [7]:
# help(cursor.execute)

In [8]:
try:
  cursor.execute('DROP TABLE test;')
except:
  pass

In [9]:
response = cursor.fetchall()
response

ProgrammingError: no results to fetch

In [10]:
connection.commit()

In [11]:
sql_create_table = '''CREATE TABLE test 
( gid serial primary key, 
  name varchar(20), 
  point geometry(POINT), 
  linestring geometry(LINESTRING),  
  polygon geometry(POLYGON), 
  multipoint geometry(MULTIPOINT), 
  multilinestring geometry(MULTILINESTRING), 
  multipolygon geometry(MULTIPOLYGON), 
  geometrycollection geometry(GEOMETRYCOLLECTION));'''

In [12]:
cursor.execute(sql_create_table)

In [13]:
response = cursor.fetchall()
response

ProgrammingError: no results to fetch

In [14]:
connection.commit()

In [15]:
cursor.execute('''
SELECT table_schema,table_name 
FROM information_schema.tables 
WHERE table_schema='public' 
ORDER BY table_schema,table_name;
''')

In [16]:
response = cursor.fetchall()

In [17]:
response

[('public', 'geography_columns'),
 ('public', 'geometry_columns'),
 ('public', 'raster_columns'),
 ('public', 'raster_overviews'),
 ('public', 'spatial_ref_sys'),
 ('public', 'test')]

### Hey hey what do you say, our table 'test' is there!

In [18]:
cursor.execute( 'SELECT * FROM test')

In [19]:
response = cursor.fetchall()
response

[]

### Looks like it's empty for now, let's create a row with data for each column (type of data)

In [20]:
cursor.execute('''
INSERT INTO test ( name, point, linestring, polygon, multipoint, multilinestring, multipolygon, geometrycollection )
VALUES (
'geometries',
ST_GeomFromText('POINT(0 0)'),
ST_GeomFromText('LINESTRING(0 0,1 1,1 2)'),
ST_GeomFromText('POLYGON((0 0,4 0,4 4,0 4,0 0),(1 1, 2 1, 2 2, 1 2,1 1))'),
ST_GeomFromText('MULTIPOINT((0 0),(1 2))'),
ST_GeomFromText('MULTILINESTRING((0 0,1 1,1 2),(2 3,3 2,5 4))'),
ST_GeomFromText('MULTIPOLYGON(((0 0,4 0,4 4,0 4,0 0),(1 1,2 1,2 2,1 2,1 1)), ((-1 -1,-1 -2,-2 -2,-2 -1,-1 -1)))'),
ST_GeomFromText('GEOMETRYCOLLECTION(POINT(2 3),LINESTRING(2 3,3 4))')
)''')

In [21]:
response = cursor.fetchall()
response

ProgrammingError: no results to fetch

In [22]:
connection.commit()

### Let's make sure the data is there by querying the dsatabase since the last commit

In [23]:
cursor.execute('''
SELECT name, St_AsText(point), St_AsText(linestring), St_AsText(polygon), St_AsText(multipoint), 
St_AsText(multilinestring), St_AsText(multipolygon), St_AsText(geometrycollection) 
FROM test'''
)

In [24]:
response = cursor.fetchall()

In [25]:
response

[('geometries',
  'POINT(0 0)',
  'LINESTRING(0 0,1 1,1 2)',
  'POLYGON((0 0,4 0,4 4,0 4,0 0),(1 1,2 1,2 2,1 2,1 1))',
  'MULTIPOINT(0 0,1 2)',
  'MULTILINESTRING((0 0,1 1,1 2),(2 3,3 2,5 4))',
  'MULTIPOLYGON(((0 0,4 0,4 4,0 4,0 0),(1 1,2 1,2 2,1 2,1 1)),((-1 -1,-1 -2,-2 -2,-2 -1,-1 -1)))',
  'GEOMETRYCOLLECTION(POINT(2 3),LINESTRING(2 3,3 4))')]

### Everything seems in place

In [26]:
connection.close()